In [4]:
import os

from src.data.data_processing.parameters import Parameters
from src.data.data_processing.video2dataset import Video2Dataset
import glob
work_dir = os.getcwd()

args= { 

    "input": f"C:/Users/gbo10/Dropbox/research videos/31.12/65-31.12/GX010065.MP4",
    "output": r"C:\Users\gbo10\Dropbox\research videos\focal length calibration\exctracted",
    "start": 0,
    "end": None,
    "output_resolution": None,
    "blur_threshold":800,
    "distance_threshold":50,
    "black_ratio_threshold": None,
    "pixel_black_threshold": None,
    "use-srt": None,
    "limit": None,
    "frame_format": "jpg",
    "stats_file":"C:/Users/gbo10/Videos/research/counting_research_algorithms/src/data/data_processing/stats.csv"}
 ##50 50 

video_paths = []  # Initialize an empty list to store the video paths

for video_path in glob.glob(r"C:\Users\gbo10\Dropbox\research videos\focal length calibration\GX010226_wby2Q0Mm2dynp_0_1724054047105.MP4"):
    video_paths.append(video_path)


for video_path in video_paths:
    args["input"] = video_path
    print(f'Processing video {video_path}')
    Video2Dataset2process = Video2Dataset(Parameters(args))
    Video2Dataset2process.ProcessVideo()



Processing video C:\Users\gbo10\Dropbox\research videos\focal length calibration\GX010226_wby2Q0Mm2dynp_0_1724054047105.MP4
before stats file
after stats file
C:/Users/gbo10/Videos/research/counting_research_algorithms/src/data/data_processing/stats.csv
C:/Users/gbo10/Videos/research/counting_research_algorithms/src/data/data_processing/stats.csv


In [ ]:
import cv2
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

class ImageQualityDetector:
    def __init__(self, ssim_threshold=0.8, psnr_threshold=30):
        self.ssim_threshold = ssim_threshold
        self.psnr_threshold = psnr_threshold

    def is_high_quality(self, image):
        # Convert image to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Compute SSIM and PSNR
        ssim_score = ssim(gray, gray)
        psnr_score = psnr(gray, gray)

        # Check if both scores are above thresholds
        is_high_quality = ssim_score > self.ssim_threshold and psnr_score > self.psnr_threshold

        return ssim_score, psnr_score, is_high_quality

# Example usage
detector = ImageQualityDetector(ssim_threshold=0.8, psnr_threshold=30)
image = cv2.imread('image.jpg')
ssim_score, psnr_score, high_quality = detector.is_high_quality(image)
print("SSIM Score:", ssim_score)
print("PSNR Score:", psnr_score)
print("Is high quality?", high_quality)


In [1]:
#exctract evry 5 frame of a video to a folder
import cv2
import os
video_path = r"C:\Users\gbo10\Dropbox\research videos\car-current-/23.07.2024\GX010208.MP4"
output_folder = r'C:\Users\gbo10\Dropbox\research videos\car-current-/23.07.2024/208/frames'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

cap = cv2.VideoCapture(video_path)
frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    if frame_count % 5 == 0:
        cv2.imwrite(f'{output_folder}/frame_{frame_count}.jpg', frame)

    frame_count += 1

cap.release()



In [2]:
#convert all the frames to grayscale
import cv2
import os
from tqdm import tqdm

input_folder = r'C:\Users\gbo10\Dropbox\research videos\car-current-/23.07.2024/208/frames'
output_folder = r'C:\Users\gbo10\Dropbox\research videos\car-current-/23.07.2024/208/frames_gray'

os.makedirs(output_folder, exist_ok=True)

for filename in tqdm( os.listdir(input_folder)):
    image = cv2.imread(f'{input_folder}/{filename}')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    cv2.imwrite(f'{output_folder}/{filename}', gray)



100%|██████████| 271/271 [06:52<00:00,  1.52s/it]


In [2]:
# resize images in folder to 640X640

import cv2
import os
from tqdm import tqdm

# with PIL
from PIL import Image
import os

input_folder = r'C:\Users\gbo10\OneDrive\measurement_paper_images\for fiftyone'

output_folder = r'C:\Users\gbo10\OneDrive\measurement_paper_images\for fiftyone\resized'

os.makedirs(output_folder, exist_ok=True)

for filename in tqdm(os.listdir(input_folder)):
    if filename.endswith('.jpg'):
        image = Image.open(f'{input_folder}/{filename}')
        image = image.resize((640, 640))
        image.save(f'{output_folder}/{filename}')   




100%|██████████| 43/43 [00:09<00:00,  4.46it/s]


In [ ]:
import cv2
import numpy as np
import os

def contrast_stretching(image):
    # Split the image into its R, G, and B channels
    r, g, b = cv2.split(image)

    # Apply contrast stretching to each channel
    r_stretched = stretch_channel(r)
    g_stretched = stretch_channel(g)
    b_stretched = stretch_channel(b)

    # Merge the channels back together
    stretched_image = cv2.merge((r_stretched, g_stretched, b_stretched))
    return stretched_image

def stretch_channel(channel):
    # Find the minimum and maximum pixel values
    min_val = np.min(channel)
    max_val = np.max(channel)

    # Apply the contrast stretching formula
    stretched_channel = (channel - min_val) * (255 / (max_val - min_val))
    stretched_channel = np.clip(stretched_channel, 0, 255).astype(np.uint8)

    return stretched_channel

def process_images_in_folder(input_folder, output_folder):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Process each image in the input folder
    for filename in tqdm(os.listdir(input_folder)):
        if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
            # Load the image
            image_path = os.path.join(input_folder, filename)
            image = cv2.imread(image_path)

            # Apply contrast stretching
            stretched_image = contrast_stretching(image)

            # Save the stretched image to the output folder
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, stretched_image)

            print(f'Processed {filename}')

# Specify the input and output folders
input_folder = r'C:\Users\gbo10\OneDrive\measurement_paper_images\for fiftyone\resized'
output_folder =r'C:\Users\gbo10\OneDrive\measurement_paper_images\for fiftyone\resized\stretched'

# Run the batch processing
process_images_in_folder(input_folder, output_folder)

#contrast stretching with color images



